In [80]:
from typing import Tuple, List, Union, Callable
from pathlib import Path
import sys
import re
import itertools

from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader
from torchvision.transforms.v2 import ToTensor, Compose
from torchvision.datasets import VisionDataset

sys.path.append('..')
from libs import list_utils as lu
from libs import transforms as tsf
from libs import seglib, charters_htr, metrics
from model_htr import HTR_Model


## TODO:

* ensure that lines are concatenated in reading order (write a utility that may detect discrepancy between line ids and reading order)
* filter GT transcriptions before aligning
* performance measure: complete, with confusion matrix and F1, on large number of manuscripts
* compare results with alignment based on edit distance

In [4]:

class InferenceDataset( VisionDataset ):

    def __init__(self, img_path: Union[str,Path],
                 segmentation_data: Union[str,Path],
                 transform: Callable=None,
                 padding_style=None) -> None:
        """ A minimal dataset class for inference on a single charter (no transcription in the sample).

        Args:
            img_path (Union[Path,str]): charter image path

            segmentation_data (Union[Path, str]): segmentation metadata (XML or JSON)
            
            transform (Callable): Image transform.

            padding_style (str): How to pad the bounding box around the polygons, when 
                building the initial, raw dataset (before applying any transform):
                + 'median'= polygon's median value,
                + 'noise' = random noise,
                + 'zero'= 0-padding, 
                + None (default) = no padding, i.e. raw bounding box
        """

        trf = transform if transform else ToTensor()
        super().__init__(root, transform=trf )

        img_path = Path( img_path ) if type(img_path) is str else img_path
        segmentation_data = Path( segmentation_data ) if type(segmentation_data) is str else segmentation_data

        # extract line images: functions line_images_from_img_* return tuples (<line_img_hwc>: np.ndarray, <mask_hwc>: np.ndarray)
        line_extraction_func = seglib.line_images_from_img_json_files if segmentation_data.suffix == '.json' else seglib.line_images_from_img_xml_files

        line_padding_func = lambda x, m, channel_dim=2: x # by default, identity function
        if padding_style == 'noise':
            line_padding_func = tsf.bbox_noise_pad
        elif padding_style == 'median':
            line_padding_func = tsf.bbox_median_pad
        elif padding_style == 'zero':
            line_padding_func = tsf.bbox_zero_pad
        print(line_padding_func)

        self.data = []

        for (img_hwc, mask_hwc) in line_extraction_func( img_path, segmentation_data ):
            mask_hw = mask_hwc[:,:,0]
            self.data.append( { 'img': line_padding_func( img_hwc, mask_hw, channel_dim=2 ), #tsf.bbox_median_pad( img_hwc, mask_hw, channel_dim=2 ), 
                                'height':img_hwc.shape[0],
                                'width': img_hwc.shape[1],
                               } )

    def __getitem__(self, index: int):
        sample = self.data[index].copy()
        #logger.debug(f"type(sample['img'])={type(sample['img'])} with shape= {sample['img'].shape}" )
        return self.transform( sample )

    def __len__(self):
        return len(self.data)


## Collect data to be aligned

### Predicted transcription on charter image

In [5]:
# Get predicted lines, from img + segmentation file
root = Path('.')

collection_path = Path('/home/nicolas/tmp/data/fsdb_work/fsdb_sample/COLLECTIONS')
img_path = collection_path.joinpath('586c0318ce423a882dac8411c6c61978/869b3709dd67347d9d243bef546a8f2c/c87b4566ac286b5ad58fa4fe1c60eb2a.img.jpg')
segmentation_file_path = collection_path.joinpath('586c0318ce423a882dac8411c6c61978/869b3709dd67347d9d243bef546a8f2c/c87b4566ac286b5ad58fa4fe1c60eb2a.lines.gt.json')

dataset = InferenceDataset( img_path, segmentation_file_path,
                          transform = Compose([ ToTensor(),
                                                tsf.ResizeToHeight(128,2048),
                                                tsf.PadToWidth(2048),]),
                          padding_style='median')
model = HTR_Model.load('../model_koenigsfelden+nuremberg_letterbooks-2014.12.15.mlmodel')

/home/nicolas/.pyenv/versions/3.8.19/lib/python3.8/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


<function bbox_median_pad at 0x7c329fcff430>


/home/nicolas/graz/htr/vre/ddpa_htr/notebooks/../model_htr.py:255: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(file_name, map_location="cpu")


In [21]:
predictions = []
for line, sample in enumerate(DataLoader(dataset,batch_size=1)):
    predicted_strings, line_scores = model.inference_task( sample['img'], sample['width'])
    predictions.append({'line_id': line, 'transcription': predicted_strings[0], 'scores': lu.flatten(line_scores.tolist())})

In [23]:
transcriptions_pred, scores = ( [ l[k] for l in predictions] for k in ('transcription', 'scores'))

In [78]:
transcriptions_pred

['Wir, Leupolt von gots gnaden hertzog ze O?sterreich, ze Steyr, ze Kernden und ze Krain, graf ze Tyrol et?u0020cetera, mbieten den erbern, geist-',
 'lichen, unsern lieben, andechtigen, dem provincial . . dem custer dem sichter und allen andern geistlichen amptluten minner',
 'bruder ordens, die des klosters ze Kunigsveld gewaltig sind, gegenwurtigen und kumftigen, den diser brief getzaigt wirt, uns',
 'gnad und alles gu?t. . wir bitten er alle und ewer yeglichen besunder und begern auch ernstlich, daz ir erun fleiz und ernste',
 'dartzu keret, damit das egenanten kloster unster stift in geistlicher zu?chtr, ordnung und fride beleiben und bestande welhe kloster-',
 'frowen daselbs sich aber dawider mit frevel, setzten und darinn nicht gehorsam sein wolten, daz ir die denn straffet nach eiers',
 'ordens gesetz und rechten und dawan derselben klosterfrowen adel oder wer frunde nicht schonet. . davon emphelhen wir',
 'dem edeln, unserm lieben oheini, gras Hannsen von Habspurg, unserm lan

### Ground truth transcriptions

In [159]:
# Get GT transcriptions
page_path = re.sub(r'\..+$', r'.xml', str(img_path))
samples = charters_htr.ChartersDataset.extract_lines_from_pagexml(page_path)
transcriptions_gt = [ s['transcription'] for s in samples]
print("GT transcriptions =", len(transcriptions_gt))


GT transcriptions = 14


## Computing alignment

In [160]:
# concatenate transcriptions for each set
transcriptions_gt_cat = ''.join(transcriptions_gt)
transcriptions_pred_cat = ''.join(transcriptions_pred)
# compute positions of line breaks in pred. (it is an offset in the string. Eg. 12 means 'after substring [0..11]
line_break_offsets_pred = list(itertools.accumulate( len(tr) for tr in transcriptions_pred ))[:-1]
line_break_offsets_pred
print("Breaks in predicted strings =", len(line_break_offsets_pred))

Breaks in predicted strings = 13


In [125]:
align_pred, align_gt = metrics.align_lcs( transcriptions_pred_cat, transcriptions_gt_cat )
#''.join([ transcriptions_pred_cat[i] for i in align_pred ])

In [162]:
def closest( tbl, val ):
    for i in range(val):
        if val-i in tbl.keys():
            #print("Closest=",val-i, end=" ")
            return val-i
            
lcs_translation_table = { p:g for (p,g) in zip( align_pred, align_gt ) }
#transcriptions_gt_segmented = []
#last_offset = 0
line_break_offsets_gt_segmented = []
for offset in line_break_offsets_pred:
    print(transcriptions_pred_cat[offset], end=' ')
    # find closest index in LCS-pred. Before or after? Depends on how
    # likely characters at SOL and EOL respectively are included in the LCS.
    lcs_i_pred = closest( lcs_translation_table, offset-1)
    lcs_i_gt = lcs_translation_table[lcs_i_pred]
    line_break_offsets_gt_segmented.append( lcs_i_gt+1 )
print("Breaks in segmented GT strings =", len(line_break_offsets_gt_segmented))

def split_on_offsets( string, offsets ):
    if not offsets:
        return string
    if not string:
        return []
    output = []
    last_offset = 0
    for i in offsets:
        output.append( string[last_offset:i] )
        last_offset = i
    output.append( string[last_offset:] )
    return output


l b g d f o d u z o w k g Breaks in segmented GT strings = 13


In [158]:
'@'.join(transcriptions_pred)
print("Length:", len( transcriptions_pred))

Length: 14


In [165]:
gt_segmented = split_on_offsets(transcriptions_gt_cat, line_break_offsets_gt_segmented)
'@'.join( gt_segmented )
print("Length:", len( gt_segmented))

Length: 14


In [129]:
#''.join([transcriptions_pred_cat[i] for i in align_pred ])

## Evaluation IoU

In [166]:
print(transcriptions_gt, "len=", len(transcriptions_gt))


['Wir, Leupolt, von gots gnaden hertzog ze Oͤsterreich, ze Steyr, ze Keͣrnden und ze Krain, graf ze Tyrol et\\u0020cetera, ✳ embieten den erbern, geist¬', 'lichen, ùnsern lieben, andêchtigen, ✳ dem provincial, ✳ dem custer, ✳ dem bichter und allen andern geistlichen amptlùten, Minner¬', 'bruder ordens, die des klosters ze Kùnigsveld gewaltig sind, gegenwùrtigen und kùmftigen, den diser brief getzaigt wirt, ùnser', 'gnad und alles guͤt. ✳ wir bitten ew alle und ewr yeglichen besunder und begern auch ernstlich, daz ir ewern fleizz und ernste', 'dartzu keret, damit das egenante kloster, unserr stift, in geistlicher zucht, ordnung und fride beleibe und bestande. ✳ welhe kloster¬', 'frowen daselbs sich aber dawider mit freͣvel setzten und darinn nicht gehorsam sein wolten, daz ir die denn straffet nach ewers', 'ordens gesetz und rechten ✳ und daran derselben klosterfrowen adel oder irer freunde nicht schonet. ✳ davon emphelhen wir', 'dem edeln, unserm lieben oͤheim, graf Hannsen von Habspur

In [167]:
print(gt_segmented, "len=", len(gt_segmented))

['Wir, Leupolt, von gots gnaden hertzog ze Oͤsterreich, ze Steyr, ze Keͣrnden und ze Krain, graf ze Tyrol et\\u0020cetera, ✳ embieten den erbern, geist', '¬lichen, ùnsern lieben, andêchtigen, ✳ dem provincial, ✳ dem custer, ✳ dem bichter und allen andern geistlichen amptlùten, Minner', '¬bruder ordens, die des klosters ze Kùnigsveld gewaltig sind, gegenwùrtigen und kùmftigen, den diser brief getzaigt wirt, ùns', 'ergnad und alles guͤt. ✳ wir bitten ew alle und ewr yeglichen besunder und begern auch ernstlich, daz ir ewern fleizz und ernste', 'dartzu keret, damit das egenante kloster, unserr stift, in geistlicher zucht, ordnung und fride beleibe und bestande. ✳ welhe kloster', '¬frowen daselbs sich aber dawider mit freͣvel setzten und darinn nicht gehorsam sein wolten, daz ir die denn straffet nach ewers', 'ordens gesetz und rechten ✳ und daran derselben klosterfrowen adel oder irer freunde nicht schonet. ✳ davon emphelhen wir', 'dem edeln, unserm lieben oͤheim, graf Hannsen von Habspur

In [168]:

# (chars shared) / Union - segmentation is done on encoded strings, for easy set computation
line_break_offsets_gt = list(itertools.accumulate( len(tr) for tr in transcriptions_gt ))[:-1]
positions = list(range(len(transcriptions_gt_cat)))
gt_sets =  [ set(s) for s in split_on_offsets( positions, line_break_offsets_gt )]
gt_segmented_sets = [ set(s) for s in split_on_offsets(positions, line_break_offsets_gt_segmented)]

print('Lists have {} and {} elements, respectively.'.format( len(transcriptions_gt), len(gt_segmented_sets)))
for l in range(len(gt_sets)):
    intersection = gt_sets[l].intersection(gt_segmented_sets[l])
    union = gt_sets[l].union(gt_segmented_sets[l])
    print("IoU =", len(intersection)/len(union))

Lists have 14 and 14 elements, respectively.
IoU = 0.9932885906040269
IoU = 0.9846153846153847
IoU = 0.9763779527559056
IoU = 0.984375
IoU = 0.9925373134328358
IoU = 0.9921875
IoU = 1.0
IoU = 0.992
IoU = 0.9923076923076923
IoU = 1.0
IoU = 0.9921875
IoU = 0.9918032786885246
IoU = 0.9830508474576272
IoU = 0.9809523809523809
